# The Solow Model with Alternative Specifications for Human Capital

This project aims to solve the Solow Model with Alternative Specifications for Human Capital. 
To accomplish this, I present the Solow Model with Technology and extend the model to account for human capital.
With the extended model, I show both analytical and numerical solutions. I also visualize the results and analyze their dependence on the model's parameters.
Having examined the model, I explore an alternative specification for human capital, considering a model with distinct groups of workers, one highly educated and the other unskilled.


Imports and set magics:

In [40]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The Solow Model

Introducing the Solow Model...

## The Solow Model with Human Capital

Human capital is a measure of a worker's productivity due to factors such as education, intelligence, and perseverance. <br>

The Solow Model with Human Capital:
- Equations

A short explanation of each equation.

Analyzing the model per capita. 

Transition Equations. 

Solow Equations. 

Steady State.

Transition: <br>
$\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}(s_{K}\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}+(1-\delta)\tilde{k}_{t})$

$\tilde{h}_{t+1} = \frac{1}{(1+n)(1+g)}(s_{H}\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}+(1-\delta)\tilde{h}_{t})$

$\tilde{k}_{t+1}-\tilde{k}_{t} = \frac{1}{(1+n)(1+g)}(s_{K}\tilde{k}_{t}^{\alpha}h_{t}^{\varphi}-(n+g+\delta+ng)\tilde{k}_{t})$

$\tilde{h}_{t+1}-\tilde{h}_{t} = \frac{1}{(1+n)(1+g)}(s_{H}\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\varphi}-(n+g+\delta+ng)\tilde{h}_{t})$



$\tilde{k}^{\ast} = \left(\frac{s_K^{1-\varphi}s_H^{\varphi}}{n + g + \delta + ng}\right)^{\frac{1}{1-\alpha-\varphi}}$

$\tilde{h}^{\ast} = \left(\frac{s_K^{\alpha}s_H^{1-\alpha}}{n + g + \delta + ng}\right)^{\frac{1}{1-\alpha-\varphi}}$

I try to solve for the steady state of capital and human capital using the transition equations and `SymPy`. <br>
To solve for the steady state, I define the parameters as symbols, formulate steady-state transition equations, and try to solve the system of equations algebraically. 

In [41]:
# define parameters as symbols
k, h, s_K, s_H, n, g, delta, alpha, varphi = sm.symbols('k, h, s_K, s_H, n, g, delta, alpha, varphi')

# formulate steady-state transition equations
ss_trans_k = sm.Eq(k,(1/((1 + n) * (1 + g))) * (s_K * k**alpha * h**varphi + (1 - delta)*k))
ss_trans_h = sm.Eq(h,(1/((1 + n) * (1 + g))) * (s_H * k**alpha * h**varphi + (1 - delta)*h))

# solve the system of equations algebraically
try:
    ss_sol = sm.solve([ss_trans_k, ss_trans_h], k, h)
    print(ss_sol[0])
except (NotImplementedError) as e:
    print(f'Unable to solve the system of equations algebraically: {e}')

Unable to solve the system of equations algebraically: could not solve -k*(1 - delta) + k*(g + 1)*(n + 1) - k**alpha*s_K*(((delta + g*n + g + n)/(k**alpha*s_H))**(1/(varphi - 1)))**varphi


Due to the limitations of `SymPy` in solving the given system of equations algebraically, I use an alternative approach. I can find the solutions by utilizing the steady-state equations for capital and human capital as defined in "Introducing Advanced Macroeconomics" by Sørensen and Whitta-Jacobsen. This involves creating lambdified expressions for the equations and using them to compute the steady-state values of capital and human capital.

In [42]:
# define steady-state equations for capital and human capital
ss_eq_k = ((s_K**(1-varphi) * s_H**varphi)/(n + g + delta + n * g))**(1/(1 - alpha - varphi))
ss_eq_h = ((s_K**(alpha) * s_H**(1-alpha))/(n + g + delta + n * g))**(1/(1 - alpha - varphi))

# create a single function that returns both steady-state values
ss_funcs = sm.lambdify((s_K, s_H, n, g, delta, alpha, varphi), (ss_eq_k, ss_eq_h))

# assign parameter values
s_K_val = 0.3
s_H_val = 0.2
n_val = 0.02
g_val = 0.02
delta_val = 0.1
alpha_val = 1/3
varphi_val = 1/3

# call the function with the parameter values
ss_eq_k_func, ss_eq_h_func = ss_funcs(s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val)

# print the steady-state values for capital and human capital
print(f'Analytical solution:\n\
There are {ss_eq_k_func:.3f} units of capital, and {ss_eq_h_func:.3f} units of human capital in steady state')

Analytical solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state


Now, that I have an analytical solution independent of any algorithm's initial values or tolerance, I can employ an optimizer to determine the steady-state values for capital and human capital and compare the results to the analytical solution. 

In [47]:
# define a function to optimize
def num_ss(variables, s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val):
    k, h = variables
    num_ss_trans_k = k - (1 / ((1 + n_val) * (1 + g_val))) * (s_K_val * k**alpha_val * h**varphi_val + (1 - delta_val) * k)
    num_ss_trans_h = h - (1 / ((1 + n_val) * (1 + g_val))) * (s_H_val * k**alpha_val * h**varphi_val + (1 - delta_val) * h)
    return [num_ss_trans_k, num_ss_trans_h]

# initial guess
initial_guess = [3, 3]

# solve the function for steady state
result = optimize.root(fun = num_ss, x0 = initial_guess, args = (s_K_val, s_H_val, n_val, g_val, delta_val, alpha_val, varphi_val), method = 'hybr')

# save the steady state values
num_ss_k, num_ss_h = result.x

# print the steady-state values for capital and human capital
print(f'Numerical Solution:\n\
There are {num_ss_k:.3f} units of capital, and {num_ss_h:.3f} units of human capital in steady state')



Numerical Solution:
There are 6.504 units of capital, and 4.336 units of human capital in steady state


I need to make a callback in optimize root where I check for which values the numerical solution converges. 

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 